In [1]:
import os
import sys

import pandas as pd
import numpy as np
## from pyhive import hive
import threshold_build_service as tbs

# %% Import and Pre-process Data

# Read csv data
df = pd.read_csv('5g_home_web_seconds_per_page_v1.csv')

C:\Users\sindh\Desktop\web_latest\threshold_build_service.py:20: UserWarning: 
The dash_core_components package is deprecated. Please replace
`import dash_core_components as dcc` with `from dash import dcc`
  import dash_core_components as dcc
C:\Users\sindh\Desktop\web_latest\threshold_build_service.py:21: UserWarning: 
The dash_html_components package is deprecated. Please replace
`import dash_html_components as html` with `from dash import html`
  import dash_html_components as html
C:\Users\sindh\Desktop\web_latest\threshold_build_service.py:25: UserWarning:


The dash_table package is deprecated. Please replace
`import dash_table` with `from dash import dash_table`

Also, if you're using any of the table format helpers (e.g. Group), replace 
`from dash_table.Format import Group` with 
`from dash.dash_table.Format import Group`



In [2]:
df = pd.read_csv('5g_home_web_seconds_per_page_v1.csv')

# Filter for a single metric
df = df.loc[df.metricname == 'seconds_per_page']

# Filter for the appropriate typology
df = df.loc[df.primary_high_level_reason == 'all']

# Rename the main column
df.rename(columns={'value':'seconds_per_page'}, inplace=True)

# Remove nulls
print(len(df))
df = df.loc[df.metricname.notnull()]
print(len(df))

# %% Instantiate Metadata (class object for threshold build)
seconds_per_page_object = tbs.Threshold_Build(
    metricname = 'seconds_per_page', 
    channel = '5g_home', 
    input_df = df.copy(deep=True)
)

# %% Analyze exisiting relationship: Plot Bivariate
# bins = [i for i in range(-7200,-9,60)]

seconds_per_page_object.plot_churn(
    bins = [i for i in range(3,50,1)]
)

48
48


In [3]:
# %% Explicitly record the relationship to churn
seconds_per_page_object.relationship_to_churn = 'positive'

seconds_per_page_object.input_df['seconds_per_page'] = seconds_per_page_object.input_df['seconds_per_page'] * (-1)

# %% Split the input dataframe
seconds_per_page_object.split = (3, None)

# %% Create universe of potential thresholds

seconds_per_page_object.create_3(
    list_of_thresholds = [-i for i in range(3, 50, 1)]
)

# %% Establish cutoffs for each potential threshold
seconds_per_page_object.create_1_5(
    ## think of argument name for how many percentiles to loop through
)

seconds_per_page_object.create_2_4(
    ## create argument for how many potential 2s and 4s we cycle through
)


# %% Create bins
seconds_per_page_object.create_bins(
        input_df = seconds_per_page_object.cutoff_universe_df,
        bin_definitions = {
                'bin_1':['1','btwn_1_2'],
                'bin_2':['2','btwn_2_3'],
                'bin_3':['3','btwn_3_4'],
                'bin_4':['4','btwn_4_5','5']
        }
)

# %% Validity Checks
seconds_per_page_object.cutoff_validity_checks(
    num_bins = 4,
    ordered_bin_suffixes = [
        'bin_1',
        'bin_2',
        'bin_3',
        'bin_4'], 
    check_monotonic = True, 
    check_pop_size = True,
    min_pop_size = (sum(seconds_per_page_object.input_df.events) * 0.01)
)

# Looking at the number of valid curves
print('Total Completely Valid Cutoffs:',
    sum((seconds_per_page_object.cutoff_universe_df['is_monotonic'] == 'Y') &
    (seconds_per_page_object.cutoff_universe_df['is_valid_bin_size'] == 'Y')),
    '\nTotal Monotonic Cutoffs:',
    sum((seconds_per_page_object.cutoff_universe_df['is_monotonic'] == 'Y')),
    '\nTotal Cutoffs with Valid Bin Sizes:',
    sum((seconds_per_page_object.cutoff_universe_df['is_valid_bin_size'] == 'Y'))
)


# %% Define ranking columns for shortlisting the best curves
print('Ranking using good vs bad')
seconds_per_page_object.define_ranking(
    ordered_bin_suffixes = [
        'bin_1',
        'bin_2',
        'bin_3',
        'bin_4'],
    ranking_formula = 'good_v_bad',
    ranking_column = 'good_v_bad_rank',
    ranking_method = 'dense',
    rounding_factor = 3
)

print('Ranking using dif between extremes')
seconds_per_page_object.define_ranking(
    ordered_bin_suffixes = [
        'bin_1',
        'bin_2',
        'bin_3',
        'bin_4'],
    ranking_formula = 'dif_in_extremes',
    ranking_column = 'dif_in_extremes_rank',
    ranking_method = 'dense',
    rounding_factor = 4
)

print('Ranking using pairwise ratio')
seconds_per_page_object.define_ranking(
    ordered_bin_suffixes = [
        'bin_1',
        'bin_2',
        'bin_3',
        'bin_4'],
    ranking_formula = 'pairwise_ratio',
    ranking_column = 'pairwise_ratio_rank',
    ranking_method = 'dense',
    rounding_factor = 1
)

print('Ranking using sum of square errors')
seconds_per_page_object.define_ranking(
    ordered_bin_suffixes = [
        'bin_1',
        'bin_2',
        'bin_3',
        'bin_4'],
    ranking_formula = 'sum_square_errors',
    ranking_column = 'sum_square_errors_rank',
    ranking_method = 'dense',
    rounding_factor = None      # Does not round the squared errors
)

# %% Saving curve dataframe to csv for inspection and manual curve selection
seconds_per_page_object.cutoff_universe_df.sort_values(
    by = [
        'is_monotonic',
        'is_valid_bin_size',
        'pairwise_ratio_rank',
        'dif_in_extremes_rank',
        'sum_square_errors_rank'
    ],
    ascending = [
        False,
        False,
        True,
        True,
        True
    ],
    inplace = True
)

# Save sorted dataframe as csv
seconds_per_page_object.cutoff_universe_df.to_csv('seconds_per_page_cutoffs.csv')

Resetting result_table
Cutoff percent: 0.01
Cutoff percent: 0.02
Cutoff percent: 0.03
Cutoff percent: 0.04
Cutoff percent: 0.05
Cutoff percent: 0.06
Cutoff percent: 0.07
Cutoff percent: 0.08
Cutoff percent: 0.09
Cutoff percent: 0.1
Threshold: -3
No possible cutoffs to choose from for cutoff 4
Size of filtered cartesian df: 0
No cutoffs found that were monotonic between 2 and 4 for threshold: -3
Monotonic cutoffs for 2 but not 4 were found for threshold: -3
Threshold: -4
No possible cutoffs to choose from for cutoff 4
Size of filtered cartesian df: 0
No cutoffs found that were monotonic between 2 and 4 for threshold: -4
Monotonic cutoffs for 2 but not 4 were found for threshold: -4
Threshold: -5
Size of filtered cartesian df: 0
No cutoffs found that were monotonic between 2 and 4 for threshold: -5
We have valid 2s and 4s, but no monotonicty between them for threshold: -5
Threshold: -6
Size of filtered cartesian df: 0
Big table is empty... Initializing
Size of big table: 72
This conditio

Size of filtered cartesian df: 5
No cutoffs found that were monotonic between 2 and 4 for threshold: -48
Monotonic cutoffs for 4 but not 2 were found for threshold: -48
Threshold: -49
No possible cutoffs to choose from for cutoff 2
Size of filtered cartesian df: 5
No cutoffs found that were monotonic between 2 and 4 for threshold: -49
Monotonic cutoffs for 4 but not 2 were found for threshold: -49
Remove old monotonic flag...
Remove old monotonic flag...Done.
Checking each row for monotonicity...
Checking each row for monotonicity...Done.
Remove old minimum population flag...
Remove old minimum population flag...Done.
Checking each row for minimum population criteria...
Checking each row for minimum population criteria...Done.
Total Completely Valid Cutoffs: 2722 
Total Monotonic Cutoffs: 2843 
Total Cutoffs with Valid Bin Sizes: 8180
Ranking using good vs bad
Ranking using dif between extremes
Ranking using pairwise ratio
Ranking using sum of square errors


In [4]:
# %% Establish best threshold (pick final candidate)

# Top row is the winner
best_seconds_per_page_curve = seconds_per_page_object.cutoff_universe_df.iloc[0,:]

#%%
# Assign cutoffs from the winner to object attributes
seconds_per_page_object.best_threshold = best_seconds_per_page_curve['threshold'] * (-1)
print('threshold:', seconds_per_page_object.best_threshold)

seconds_per_page_object.best_cutoff_1 =  best_seconds_per_page_curve['cutoff_1'] * (-1)
print('cutoff 1:', seconds_per_page_object.best_cutoff_1)

seconds_per_page_object.best_cutoff_2 =  best_seconds_per_page_curve['cutoff_2'] * (-1)
print('cutoff 2:', seconds_per_page_object.best_cutoff_2)

seconds_per_page_object.best_cutoff_4 =  best_seconds_per_page_curve['cutoff_4'] * (-1)
print('cutoff 4:', seconds_per_page_object.best_cutoff_4)

seconds_per_page_object.best_cutoff_5 =  best_seconds_per_page_curve['cutoff_5'] * (-1)
print('cutoff 5:', seconds_per_page_object.best_cutoff_5)

threshold: 35
cutoff 1: 43
cutoff 2: 42
cutoff 4: 15
cutoff 5: 3
